<a href="https://colab.research.google.com/github/Mukiti-Ryan/Credit-Card-Churn-Prediction/blob/main/Credit_Card_Churn_Prediction_(TFDF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests -q
!pip install keras-tuner -q

import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf
import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.4 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/credit_card_churn.csv')
df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
           'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
        axis = 1, inplace = True)

df.shape

(10127, 21)

In [ ]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [ ]:
label = "Attrition_Flag"

classes = list(df[label].unique())
print(f"Label Classes: {classes}")

df[label] = df[label].map(classes.index)

Label Classes: ['Existing Customer', 'Attrited Customer']


In [ ]:
np.random.seed(1)

test_indices = np.random.rand(len(df)) < 0.1
df_train = df[~test_indices]
df_test = df[test_indices]

print("Training example: ", len(df_train))

print("Testing example: ", len(df_test))

Training example:  9114
Testing example:  1013


In [ ]:
tf_train = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label = label)
model = tfdf.keras.CartModel()
model.fit(tf_train)

Use /tmp/tmps5i5zcb7 as temporary training directory
Reading training dataset...


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Training dataset read in 0:00:07.095497. Found 9114 examples.
Training model...
Model trained in 0:00:00.340302
Compiling model...


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth = 10)

In [ ]:
model.compile("accuracy")
print("Train evaluation: ", model.evaluate(tf_train, return_dict = True))

tf_test = tfdf.keras.pd_dataframe_to_tf_dataset(df_test, label = label)
print("Test evaluation: ", model.evaluate(tf_test, return_dict = True))

10/10 [==============================] - 1s 37ms/step - loss: 0.0000e+00 - accuracy: 0.9443
Train evaluation:  {'loss': 0.0, 'accuracy': 0.9442615509033203}
2/2 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 0.9339
Test evaluation:  {'loss': 0.0, 'accuracy': 0.9338598251342773}


In [ ]:
def build_model(hp):
  model = tfdf.keras.CartModel(
      min_examples = hp.Choice("min_examples", [1, 2, 4, 10]),
      validation_ratio = hp.Choice("validation_ratio", [0.0, 0.05, 0.10]),
  )
  model.compile("accuracy")
  return model

tuner = kt.RandomSearch(
    build_model,
    objective = "val_accuracy",
    max_trials = 10,
    directory = "/tmp/tuner",
    project_name = "tune_cart")

tuner.search(x = tf_train, validation_data = tf_test)
best_model = tuner.get_best_models()[0]

print("Best hyperparameters: ", tuner.get_best_hyperparameters()[0].values)

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.9368213415145874

Best val_accuracy So Far: 0.9407699704170227
Total elapsed time: 00h 00m 29s
Use /tmp/tmp8a7wabkz as temporary training directory
Best hyperparameters:  {'min_examples': 1, 'validation_ratio': 0.0}


In [ ]:
model = tfdf.keras.CartModel(min_examples = 2, validation_ratio = 0.0)
model.fit(tf_train)

model.compile("accuracy")
print("Test evaluation: ", model.evaluate(tf_test, return_dict = True))

Use /tmp/tmpjds2qb18 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.752968. Found 9114 examples.
Training model...
Model trained in 0:00:00.099486
Compiling model...
Model compiled.
2/2 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 0.9220
Test evaluation:  {'loss': 0.0, 'accuracy': 0.9220138192176819}


In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth = 10)